Adapted from `rlcard/examples/leduc_holdem_dqn_pytorch.py`

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('./rlcard'))

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
import torch
import rlcard
import numpy as np
from DQNAgent import DQNAgent
from rlcard.agents import RandomAgent, CFRAgent
from rlcard.utils import set_global_seed, tournament
from rlcard.utils import Logger

In [4]:
set_global_seed(0)

In [5]:
eval_env = rlcard.make('leduc-holdem', config={'seed': 0})

In [6]:
actions = eval_env.actions

In [7]:
dqn_agent = DQNAgent(
    eval_env.action_num,
    eval_env.state_shape[0],
    memory=2_000_000,
    lr=0.1,
    batch_size=128,
    update_every=128,
    eps_start=0.12,
    target_update=300,
    hidden_neurons=[64]
)

dqn_agent.load('models/leduc_holdem_dqn_million/step-1921384.pt')

Indices 0-2 represent the hero's hole card
* 0 - Jack
* 1 - Queen
* 2 - King

Indices 3-5 represent the public card
* 3 - Jack
* 4 - Queen
* 5 - King

Indices 6-20 represent the number of chips the hero has bet
* 6 + num chips

Indices 21-35 represent the number of chips the villain has bet
* 21 + num chips

In [8]:
def get_obs(hole_card, public_card, hero_bet, villain_bet):
    obs = np.zeros(36)
    card_to_index = { 'J': 0, 'Q': 1, 'K': 2 }
    obs[card_to_index[hole_card]] = 1
    
    if public_card:
        obs[card_to_index[public_card] + 3] = 1
    
    obs[hero_bet + 6] = 1
    obs[villain_bet + 21] = 1
    return obs

def get_actions(moves):
    return [actions.index(a) for a in moves]

## First round

### Jack hole card

First to act or facing check

In [9]:
state = { 'obs': get_obs('J', '', 1, 1), 'legal_actions': get_actions(['raise', 'fold', 'check']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'check'

Facing bet

In [10]:
state = { 'obs': get_obs('J', '', 1, 3), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'call'

### Queen hole card

First to act or facing check

In [11]:
state = { 'obs': get_obs('Q', '', 1, 1), 'legal_actions': get_actions(['raise', 'fold', 'check']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'check'

Facing bet

In [12]:
state = { 'obs': get_obs('Q', '', 1, 3), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'call'

### King hole card

First to act or facing check

In [13]:
state = { 'obs': get_obs('K', '', 1, 1), 'legal_actions': get_actions(['raise', 'fold', 'check']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'raise'

Facing bet

In [14]:
state = { 'obs': get_obs('K', '', 1, 3), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'raise'

Bet and then facing raise

In [15]:
state = { 'obs': get_obs('K', '', 3, 5), 'legal_actions': get_actions(['call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'call'

## Second round

### Jack hole card

#### Jack public card

- First round: check-check
- Second round: first to act or facing check

In [16]:
state = { 'obs': get_obs('J', 'J', 1, 1), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'raise'

- First round: check-check
- Second round: facing bet

In [17]:
state = { 'obs': get_obs('J', 'J', 1, 5), 'legal_actions': get_actions(['raise', 'call', 'fold']) }

index, _ = dqn_agent.eval_step(state)
actions[index]

'call'

- First round: check-check
- Second round: re-raised

In [18]:
state = { 'obs': get_obs('J', 'J', 5, 9), 'legal_actions': get_actions(['call', 'fold']) }

index, _ = dqn_agent.eval_step(state)
actions[index]

'call'

- First round: called bet
- Second round: first to act or facing check

In [19]:
state = { 'obs': get_obs('J', 'J', 3, 3), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'raise'

- First round: called bet
- Second round: facing bet

In [20]:
state = { 'obs': get_obs('J', 'J', 3, 7), 'legal_actions': get_actions(['raise', 'call', 'fold']) }

index, _ = dqn_agent.eval_step(state)
actions[index]

'raise'

- First round: called bet
- Second round: re-raised

In [21]:
state = { 'obs': get_obs('J', 'J', 7, 11), 'legal_actions': get_actions(['call', 'fold']) }

index, _ = dqn_agent.eval_step(state)
actions[index]

'call'

#### Queen public card

- First round: check-check
- Second round: first to act or facing check

In [22]:
state = { 'obs': get_obs('J', 'Q', 1, 1), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'check'

- First round: check-check
- Second round: facing bet

In [23]:
state = { 'obs': get_obs('J', 'Q', 1, 5), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'fold'

- First round: called bet
- Second round: first to act or facing check

In [24]:
state = { 'obs': get_obs('J', 'Q', 3, 3), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'check'

- First round: called bet
- Second round: facing bet

In [25]:
state = { 'obs': get_obs('J', 'Q', 3, 7), 'legal_actions': get_actions(['raise', 'call', 'fold']) }

index, _ = dqn_agent.eval_step(state)
actions[index]

'fold'

#### King public card

- First round: check-check
- Second round: first to act or facing check

In [26]:
state = { 'obs': get_obs('J', 'K', 1, 1), 'legal_actions': get_actions(['raise', 'check', 'fold'])}
index, _ = dqn_agent.eval_step(state)
actions[index]

'check'

- First round: check-check
- Second round: facing bet

In [27]:
state = { 'obs': get_obs('J', 'K', 1, 5), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'fold'

- First round: called bet
- Second round: first to act or facing check

In [28]:
state = { 'obs': get_obs('J', 'K', 3, 3), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'check'

- First round: called bet
- Second round: facing bet

In [29]:
state = { 'obs': get_obs('J', 'K', 3, 7), 'legal_actions': get_actions(['raise', 'call', 'fold']) }

index, _ = dqn_agent.eval_step(state)
actions[index]

'fold'

### Queen hole card

#### Jack public card

- First round: check-check
- Second round: first to act or facing check

In [30]:
state = { 'obs': get_obs('Q', 'J', 1, 1), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'fold'

- First round: check-check
- Second round: facing bet

In [31]:
state = { 'obs': get_obs('Q', 'J', 1, 5), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'fold'

- First round: called bet
- Second round: first to act or facing check

In [32]:
state = { 'obs': get_obs('Q', 'J', 3, 3), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'check'

- First round: called bet
- Second round: facing bet

In [33]:
state = { 'obs': get_obs('Q', 'J', 3, 7), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'fold'

#### Queen public card

- First round: check-check
- Second round: first to act or facing check

In [34]:
state = { 'obs': get_obs('Q', 'Q', 1, 1), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'raise'

- First round: check-check
- Second round: facing bet

In [35]:
state = { 'obs': get_obs('Q', 'Q', 1, 5), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'call'

- First round: check-check
- Second round: re-raised

In [36]:
state = { 'obs': get_obs('Q', 'Q', 5, 9), 'legal_actions': get_actions(['call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'call'

- First round: called bet
- Second round: first to act or facing check

In [37]:
state = { 'obs': get_obs('Q', 'Q', 3, 3), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'raise'

- First round: called bet
- Second round: re-raised

In [38]:
state = { 'obs': get_obs('Q', 'Q', 7, 11), 'legal_actions': get_actions(['call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'call'

- First round: called bet
- Second round: facing bet

In [39]:
state = { 'obs': get_obs('Q', 'Q', 3, 7), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'call'

#### King public card

- First round: check-check
- Second round: first to act or facing check

In [40]:
state = { 'obs': get_obs('Q', 'K', 1, 1), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'check'

- First round: check-check
- Second round: facing bet

In [41]:
state = { 'obs': get_obs('Q', 'K', 1, 5), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'fold'

- First round: called bet
- Second round: first to act or facing check

In [42]:
state = { 'obs': get_obs('Q', 'K', 3, 3), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'check'

- First round: called bet
- Second round: facing bet

In [43]:
state = { 'obs': get_obs('Q', 'K', 3, 7), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'fold'

### King hole card

#### Jack public card

- First round: called bet
- Second round: first to act or facing check

In [44]:
state = { 'obs': get_obs('K', 'J', 3, 3), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'check'

- First round: called bet
- Second round: facing bet

In [45]:
state = { 'obs': get_obs('K', 'J', 3, 7), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'fold'

- First round: called raise
- Second round: first to act or facing check

In [46]:
state = { 'obs': get_obs('K', 'J', 5, 5), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'check'

- First round: called raise
- Second round: facing bet

In [47]:
state = { 'obs': get_obs('K', 'J', 5, 9), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'fold'

#### Queen public card

- First round: called bet
- Second round: first to act or facing check

In [48]:
state = { 'obs': get_obs('K', 'Q', 3, 3), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'check'

- First round: called bet
- Second round: facing bet

In [49]:
state = { 'obs': get_obs('K', 'Q', 3, 7), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'call'

- First round: called raise
- Second round: first to act or facing check

In [50]:
state = { 'obs': get_obs('K', 'Q', 5, 5), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'check'

- First round: called raise
- Second round: facing bet

In [51]:
state = { 'obs': get_obs('K', 'Q', 5, 9), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'call'

#### King public card

- First round: called bet
- Second round: first to act or facing check

In [52]:
state = { 'obs': get_obs('K', 'K', 3, 3), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'raise'

- First round: called bet
- Second round: facing bet

In [53]:
state = { 'obs': get_obs('K', 'K', 3, 7), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'raise'

- First round: called bet
- Second round: facing re-raise

In [54]:
state = { 'obs': get_obs('K', 'K', 7, 11), 'legal_actions': get_actions(['call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'call'

- First round: called raise
- Second round: first to act or facing check

In [55]:
state = { 'obs': get_obs('K', 'K', 5, 5), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'raise'

- First round: called raise
- Second round: facing re-raise

In [56]:
state = { 'obs': get_obs('K', 'K', 9, 13), 'legal_actions': get_actions(['call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'call'

- First round: called raise
- Second round: facing bet

In [57]:
state = { 'obs': get_obs('K', 'K', 5, 9), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'call'